# Packages

In [1]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Open Nighttime Lights'

# Sentinel-2

Sentinel-2 satellites (there are 2) are part of the European Space Agency’s (ESA) Copernicus system and contain the MultiSpectral Instrument (MSI), which provides hi-resolution imagery that is useful for land use monitoring.

As with other datasets in Google Earth Engine (GEE), more details and links to resources are available on the collection’s landing pages.

What is important for us is that it has several image bands across the optical electromagnetic spectrum that will be useful feature for classifying land use, particular that of built-up areas. Let’s explore this data source and get sense of it’s resolution and attributes. For visualization we’ll look at just the Red, Green and Blue channels and clip around Nepal.

In [2]:
# Get Nepal boundary
aoi = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', 'Nepal')).geometry()

# Sentinel-2 image filtered on 2019 and on Nepal
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate('2019-01-01', '2019-12-31').filterBounds(aoi).median().divide(10000)

rgb = ['B4', 'B3', 'B2']

# Set some thresholds
rgbViz = {
    'min': 0,
    'max': 0.3,
    'bands': rgb
}

# Initialize our map
map1 = geemap.Map()
map1.centerObject(aoi, 7)
map1.addLayer(se2.clip(aoi), rgbViz, 'S2')
map1.addLayerControl()
map1

Map(center=[28.268224502649357, 83.97637443188768], controls=(WidgetControl(options=['position', 'transparent_…

We can see a real color image of Nepal. We reduced our Image Collection to an image representing the median of 2019 and it appears we’ve also captured some clouds around Kathmandu. We will make a cloud mask to clear the image up using Sentinel-2’s QA band.

In [3]:
def se2mask(image):
    quality_band = image.select('QA60')

    # Using the bit mask for clouds and cirrus clouds respectively
    cloudmask = 1 << 10
    cirrusmask = 1 << 11

    # We only want clear skies
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0))

    # We'll divide by 10000 to make interpreting the reflectance values easier
    return image.updateMask(mask).divide(10000)

se2 = ee.ImageCollection('COPERNICUS/S2').filterDate(
    '2019-01-01', '2019-12-31'
).filterBounds(aoi).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(se2mask).median()

# Initialize map
map2 = geemap.Map()
map2.centerObject(aoi, 7)
map2.addLayer(se2.clip(aoi), rgbViz, 'S2')
map2.addLayerControl()
map2

Map(center=[28.268224502649357, 83.97637443188768], controls=(WidgetControl(options=['position', 'transparent_…

# Global Human Settlement Layer

Identifying areas of human settlement is a large area of focus in Earth Observation and many other disciplines. The Global Human Settlement Layer (GHSL) dataset is a useful resource for understanding areas of settlement and happily for our purposes, it is available in the Google Earth Engine (GEE) catalogue.

The dataset we are particularly interested in is the GHSL “Settlement Grid” layer. The settlement grids in this dataset have been generated via the GHSL built-up areas and population grids, which are themselves derived from Landsat image collections and other sources and these layers are also available on GEE. A couple things to note.

First, there is one band with four “degrees of urbanization”:
- Inhabited areas
- Rural grid cells
- Low Density Clusters (towns and cities)
- High Density Clusters (cities)

We made a choice in framing our analysis that we are interested in the change of the Low and High Density clusters (“built up”) relative to everything else, so we will classify any pixel with values in [3,4] as “built up” and assign this 1 or not and assign it 0.

A second item worth noting is that the spatial resolution for this grid layer is 1000 meters.

In [4]:
ghsl = ee.ImageCollection('JRC/GHSL/P2016/SMOD_POP_GLOBE_V1').filter(ee.Filter.date('2015-01-01', '2015-12-31')).select('smod_code').median()

# Create a boolean mask setting anything equaling 2 (low density) or 3 (high density) as True. This will be our binary label layer
ghslbinary = ghsl.gte(2)

ghslViz = {
    'min': 0,
    'max': 3,
    'palette': ['000000', '448564', '70daa4', 'ffffff']
}
ghslbiViz = {'palette': ['000000', 'ffffff']}

map3 = geemap.Map()
map3.centerObject(aoi, 7)
map3.addLayer(ghsl.clip(aoi), ghslViz, 'Degree of Urbanization')
map3.addLayer(ghslbinary.clip(aoi), ghslbiViz, 'Built Up')
map3

Map(center=[28.268224502649357, 83.97637443188768], controls=(WidgetControl(options=['position', 'transparent_…

In the original data layer (“Degree of Urbanization”) you can see the high density patch of Kathmandu (white) as well as the low density cluster classifications that follow what appears to be major road networks and other towns (bright green). We can also see the rural areas (green) and the rest of the country (black).

In our binary mask (“Built up”), you can see how the coverage is over both low and high cluster areas. This is the layer we will use as our labels for the training data.